In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

#Helper Functions

def date_time(table_cells):
    """Extract date and time from HTML table cell"""
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """Extract booster version from HTML table cell"""
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """Extract landing status from HTML table cell"""
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """Extract payload mass from HTML table cell"""
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    """Extract column name from HTML table header"""
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)
    
    # Filter digit and empty names
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name
    


In [2]:
#Task 1: Request the Falcon 9 Launch Wiki Page

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Request the page
response = requests.get(static_url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Verify the page title
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


In [3]:
#Task 2: Extract Column Names from HTML Table Header

# Find all tables
html_tables = soup.find_all('table')

# The third table contains launch records
first_launch_table = html_tables[2]

# Extract column names
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [4]:
#Task 3: Create DataFrame by Parsing Launch Tables

# Initialize dictionary
launch_dict = dict.fromkeys(column_names)

# Remove irrelevant column
del launch_dict['Date and time ( )']

# Initialize dictionary values as empty lists
for key in launch_dict:
    launch_dict[key] = []
    
# Add new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Parse all launch tables
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # Check if first table heading is a launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
            
        # Get table elements
        row = rows.find_all('td')
        
        if flag:
            extracted_row += 1
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            
            # Booster version
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Create DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Save to CSV
df.to_csv('spacex_web_scraped.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'string'

In [7]:
# Initialize dictionary for launch data
launch_dict = {
    'Flight No.': [],
    'Date': [],
    'Time': [],
    'Version Booster': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Booster landing': []
}

# Parse all launch tables
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # Check if first table heading is a launch number
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False
            
        # Get table elements
        row = rows.find_all('td')
        
        if flag and len(row) >= 9:  # Ensure we have enough columns
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time - with error handling
            try:
                datatimelist = date_time(row[0])
                date = datatimelist[0].strip(',') if len(datatimelist) > 0 else "Unknown"
                time = datatimelist[1] if len(datatimelist) > 1 else "Unknown"
            except:
                date, time = "Unknown", "Unknown"
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            
            # Booster version - with error handling
            try:
                bv = booster_version(row[1])
                if not bv:
                    bv = row[1].a.string if row[1].a else ' '.join(row[1].strings).strip()
            except:
                bv = "Unknown"
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site - with error handling
            try:
                launch_site = row[2].a.string if row[2].a else ' '.join(row[2].strings).strip()
            except:
                launch_site = "Unknown"
            launch_dict['Launch site'].append(launch_site)
            
            # Payload - with error handling
            try:
                payload = row[3].a.string if row[3].a else ' '.join(row[3].strings).strip()
            except:
                payload = "Unknown"
            launch_dict['Payload'].append(payload)
            
            # Payload Mass - with error handling
            try:
                payload_mass = get_mass(row[4])
                if not payload_mass or payload_mass == "0":
                    payload_mass = "Unknown"
            except:
                payload_mass = "Unknown"
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit - with error handling
            try:
                orbit = row[5].a.string if row[5].a else ' '.join(row[5].strings).strip()
            except:
                orbit = "Unknown"
            launch_dict['Orbit'].append(orbit)
            
            # Customer - with error handling
            try:
                customer = row[6].a.string if row[6].a else ' '.join(row[6].strings).strip()
                if not customer:
                    customer = "Unknown"
            except:
                customer = "Unknown"
            launch_dict['Customer'].append(customer)
            
            # Launch outcome - with error handling
            try:
                launch_outcome = list(row[7].strings)[0] if row[7].strings else "Unknown"
            except:
                launch_outcome = "Unknown"
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing - with error handling
            try:
                booster_landing = landing_status(row[8]) if len(row) > 8 else "Unknown"
            except:
                booster_landing = "Unknown"
            launch_dict['Booster landing'].append(booster_landing)

# Create DataFrame from the dictionary
df = pd.DataFrame(launch_dict)

# Clean up the data
df.replace(["", "N/A", "None", "nan", "NaN"], "Unknown", inplace=True)
df.fillna("Unknown", inplace=True)

# Save to CSV
df.to_csv('spacex_web_scraped.csv', index=False)

print(f"Successfully extracted {extracted_row} launch records")
print(df.head())

Successfully extracted 121 launch records
  Flight No.             Date   Time   Version Booster Launch site  \
0          1      4 June 2010  18:45  F9 v1.07B0003.18       CCAFS   
1          2  8 December 2010  15:43  F9 v1.07B0004.18       CCAFS   
2          3      22 May 2012  07:44  F9 v1.07B0005.18       CCAFS   
3          4   8 October 2012  00:35  F9 v1.07B0006.18       CCAFS   
4          5     1 March 2013  15:10  F9 v1.07B0007.18       CCAFS   

                                Payload Payload mass Orbit Customer  \
0  Dragon Spacecraft Qualification Unit      Unknown   LEO   SpaceX   
1                                Dragon      Unknown   LEO     NASA   
2                                Dragon       525 kg   LEO     NASA   
3                          SpaceX CRS-1     4,700 kg   LEO     NASA   
4                          SpaceX CRS-2     4,877 kg   LEO     NASA   

  Launch outcome Booster landing  
0      Success\n         Failure  
1        Success         Failure  
2    